In [1]:
# Always neds to be done in Rasberry Pi
import findspark
findspark.init('/home/baxman/spark-2.4.7-bin-hadoop2.7')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('logireg').getOrCreate()

In [2]:
# Import LinReg from MLlib
from pyspark.ml.classification import LogisticRegression

In [4]:
# Set data
data = spark.read.format('libsvm').load('/home/baxman/Codes/PySpark/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Logistic_Regression/sample_libsvm_data.txt')

In [5]:
#Already set up
data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [6]:
logireg = LogisticRegression()

In [7]:
fitted_logireg = logireg.fit(data)

In [13]:
logi_reg_summary = fitted_logireg.summary

In [17]:
logi_reg_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [18]:
# Does label match precitions?
logi_reg_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947479...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194909...|[1.41321555110966...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865127001...|       1.0|
|  1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170463...|       1.0|
|  1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200654...|       1.0|
|  0.0|(692,[129,130,131...|[19.8506078990278...|[0.99999999760673...|       0.0|
|  1.0|(692,[158,159,160...|[-20.337256674834...|[1.47109814695472...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.595579753418...|[3.08850168102557...|       1.0|
|  0.0|(692,[154,155,156...|[19.2708803215614...|[0.99999999572670...|       0.0|
|  0.0|(692,[127

In [20]:
# Using evaluators -> need to split data into tran/test split

training, test =  data.randomSplit([0.7,0.3])

In [22]:
# Retrain on training

logireg2 = LogisticRegression()
fitted2 = logireg2.fit(training)

In [24]:
# Evaluating on test set
predicitions_labels = fitted2.evaluate(test)

In [27]:
predicitions_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[123,124,125...|[30.1369035922759...|[0.99999999999991...|       0.0|
|  0.0|(692,[124,125,126...|[17.9290504290184...|[0.99999998365020...|       0.0|
|  0.0|(692,[126,127,128...|[22.2124351304914...|[0.99999999977444...|       0.0|
|  0.0|(692,[126,127,128...|[24.9968626459549...|[0.99999999998606...|       0.0|
|  0.0|(692,[126,127,128...|[27.5002617814614...|[0.99999999999886...|       0.0|
|  0.0|(692,[127,128,129...|[20.9346149543363...|[0.99999999919050...|       0.0|
|  0.0|(692,[127,128,129...|[24.8457271092936...|[0.99999999998379...|       0.0|
|  0.0|(692,[151,152,153...|[25.9436451620190...|[0.99999999999459...|       0.0|
|  0.0|(692,[153,154,155...|[13.3055134502053...|[0.99999833471639...|       0.0|
|  0.0|(692,[154

In [28]:
# Explore evaluation with evaluator object

from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [30]:
evaluator = BinaryClassificationEvaluator()

In [31]:
roc = evaluator.evaluate(predicitions_labels.predictions)

In [32]:
roc

1.0

In [ ]:
# Perfect fit, but this is fake data from documentation